# Process NeuroPixels Electrophysiology data with DataJoint Elements

This notebook will walk through processing NeuroPixels array electrophysiology
data acquired with spikeGLX and processed with kilosort. While anyone can work
through this notebook to process electrophysiology data through DataJoint's
`element-array-ephys` pipeline, for a detailed tutorial about the fundamentals of
DataJoint including table types, make functions, and querying, please see the [DataJoint Tutorial](https://github.com/datajoint/datajoint-tutorials).

The DataJoint Python API and Element Array Electrophysiology offer a lot of features to support collaboration, automation, reproducibility, and visualizations.

For more information on these topics, please visit our documentation: 
 
- [DataJoint Core](https://datajoint.com/docs/core/): General principles

- DataJoint [Python](https://datajoint.com/docs/core/datajoint-python/) and
  [MATLAB](https://datajoint.com/docs/core/datajoint-matlab/) APIs: in-depth reviews of
  specifics

- [DataJoint Element Array Ephys](https://datajoint.com/docs/elements/element-array-ephys/):
  A modular pipeline for electrophysiology analysis


Let's start by importing the packages necessary to run this tutorial.

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
import datetime
import matplotlib.pyplot as plt
import numpy as np

### The Basics:

Any DataJoint workflow can be broken down into basic 3 parts:

- `Insert`
- `Populate` (or process)
- `Query`

In this demo we will:
- `Insert` metadata about an animal subject, recording session, and 
  parameters related to processing electrophysiology data.
- `Populate` tables with outputs of ephys recording data including LFPs, and spike
  sorted waveforms and units.
- `Query` the processed data from the database and plot waveform traces.

Each of these topics will be explained thoroughly in this notebook.

### Workflow diagram

This workflow is assembled from 4 DataJoint elements:
+ [element-lab](https://github.com/datajoint/element-lab)
+ [element-animal](https://github.com/datajoint/element-animal)
+ [element-session](https://github.com/datajoint/element-session)
+ [element-array-ephys](https://github.com/datajoint/element-array-ephys)

Each element declares its own schema in the database. These schemas can be imported like
any other Python package. This workflow is composed of schemas from each of the Elements
above and correspond to a module within `workflow_array_ephys.pipeline`.

The schema diagram is a good reference for understanding the order of the tables
within the workflow, as well as the corresponding table type.
Let's activate the elements and view the schema diagram.

In [ ]:
from workflow_array_ephys.pipeline import lab, subject, session, probe, ephys

In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(probe)
    + dj.Diagram(ephys)
)

### Diagram Breakdown

While the diagram above seems complex at first, it becomes more clear when it's
approached as a hierarchy of tables that **define the order** in which the
workflow **expects to receive data** in each of its tables. 

- Tables with a green, or rectangular shape expect to receive data manually
  using the `insert()` function. The tables higher up in the diagram such as
  `subject.Subject()` should be the first to receive data. This ensures data
  integrity by preventing orphaned data within DataJoint schemas. 
- Tables with a purple oval or red circle can be automatically filled with relevant data
  by calling `populate()`. For example `ephys.EphysRecording` and its part-table
  `ephys.EphysRecording.EphysFile` are both populated with
  `ephys.EphysRecording.populate()`.
- Tables connected by a solid line depend on attributes (entries) in the table
  above it.

#### Table Types

There are 5 table types in DataJoint. Each of these appear in the diagram above.

- **Manual table**: green box, manually inserted table, expect new entries daily, e.g. `Subject`, `ProbeInsertion`.  
- **Lookup table**: gray box, pre inserted table, commonly used for general facts or parameters. e.g. `Strain`, `ClusteringMethod`, `ClusteringParamSet`.  
- **Imported table**: blue oval, auto-processing table, the processing depends on the importing of external files. e.g. process of `Clustering` requires output files from kilosort2.  
- **Computed table**: red circle, auto-processing table, the processing does not
  depend on files external to the database.  
- **Part table**: plain text, as an appendix to the master table, all the part
  entries of a given master entry represent a intact set of the master entry.
  e.g. `Unit` of a `CuratedClustering`.


## Starting the workflow: Insert

### Insert entries into manual tables

To view details about a table's dependencies and attributes, use functions `.describe()`
and `.heading`, respectively.

Let's start with the first table in the schema diagram (the `subject` table) and view
the table attributes we need to insert. There are two ways you can do this: *run each
of the two cells below*

In [ ]:
subject.Subject.describe()

In [ ]:
subject.Subject.heading

The cells above show all attributes of the subject table. These are particularly useful functions if you are new to
DataJoint Elements and are unsure of the attributes required for each table. We will insert data into the
`subject.Subject` table.

In [ ]:
subject.Subject.insert1(
    dict(subject="subject5", sex="M", subject_birth_date="2020-01-04"),
    skip_duplicates=True,
)
subject.Subject()

Let's repeat the steps above for the `Session` table and see how the output varies between
`.describe` and `.heading`.

In [ ]:
session.Session.describe()

In [ ]:
session.Session.heading

The cells above show the dependencies and attributes for the `session.Session` table.
Notice that `describe` shows the dependencies of the table on upstream tables. The
`Session` table depends on the upstream `Subject` table. 

Whereas `heading` lists all the attributes of the `Session` table, regardless of
whether they are declared in an upstream table. 

Here we will demonstrate a very useful way of inserting data by assigning the dictionary
to a variable `session_key`. This variable can be used to insert entries into tables that
contain the `Session` table as one of its attributes.

In [ ]:
session_key = dict(subject="subject5", session_datetime="2023-01-01 00:00:00.000")
session.Session.insert1(session_key, skip_duplicates=True)
session.Session()

The `SessionDirectory` table locates the relevant data files in a directory path
relative to the root directory defined in your `dj.config["custom"]`. More
information about `dj.config` is provided at the end of this tutorial and is
particularly useful for local deployments of this workflow.

In [ ]:
session.SessionDirectory.describe()

In [ ]:
session.SessionDirectory.heading

In [ ]:
session.SessionDirectory.insert1(
    dict(
        **session_key,
        session_dir="subject5/session1",
    ),
    skip_duplicates=True,
)
session.SessionDirectory()

As the workflow diagram indicates, the tables in the `probe` schemas need to
contain data before the tables in the `ephys` schema accept any data. Let's
start by inserting into `probe.Probe`, a table containing metadata about a
multielectrode probe. 

In [ ]:
probe.Probe.describe()

In [ ]:
probe.Probe.heading

In [ ]:
probe.Probe.insert1(
    dict(probe="17131311651", probe_type="neuropixels 1.0 - 3B"), skip_duplicates=True
)  # this info could be achieve from neuropixels meta file.
probe.Probe()

The probe metadata is used by the downstream `ProbeInsertion` table which we
insert data into in the cells below:

In [ ]:
ephys.ProbeInsertion.describe()

In [ ]:
ephys.ProbeInsertion.heading

In [ ]:
ephys.ProbeInsertion.insert1(
    dict(
        **session_key,
        insertion_number=0,
        probe="17131311651",
    ),
    skip_duplicates=True,
)  # probe, subject, session_datetime needs to follow the restrictions of foreign keys.
ephys.ProbeInsertion()

## Populate

### Automatically populate tables

`ephys.EphysRecording` is the first table in the pipeline that can be populated automatically.
If a table contains a part table, this part table is also populated during the
`populate()` call. `populate()` takes several arguments including the a session
key. This key restricts `populate()` to performing the operation on the session
of interest rather than all possible sessions which could be a time-intensive
process for databases with lots of entries.

Let's view the `ephys.EphysRecording` and its part table
`ephys.EphysRecording.EphysFile` and populate both through a single `populate()`
call.

In [ ]:
ephys.EphysRecording.heading

In [ ]:
ephys.EphysRecording.EphysFile.heading

In [ ]:
ephys.EphysRecording()

In [ ]:
ephys.EphysRecording.EphysFile()

In [ ]:
session_key

In [ ]:
ephys.EphysRecording.populate(session_key, display_progress=True)

In [ ]:
from element_interface.utils import dict_to_uuid, find_full_path, find_root_directory

In [ ]:
key = (ephys.ProbeInsertion & "subject = 'subject5'").fetch1("KEY")

In [ ]:
key

In [ ]:
from element_array_ephys.ephys_acute import get_ephys_root_data_dir, get_session_directory

In [ ]:
ephys_pattern = "*.ap.meta"
session_dir = find_full_path(
    get_ephys_root_data_dir(), get_session_directory(key)
        )
ephys_meta_filepaths = list(session_dir.rglob(ephys_pattern))

In [ ]:
session_dir

In [ ]:
import pathlib

In [ ]:
list(session_dir.rglob(ephys_pattern))

Let's view the information was entered into each of these tables:

In [ ]:
ephys.EphysRecording()

In [ ]:
ephys.EphysRecording.EphysFile()

We're almost ready to perform image processing with `Suite2p`. An important step before
processing is managing the parameters which will be used in that step. To do so, we will
import suite2p and insert the default parameters into a DataJoint table
`ProcessingParamSet`. This table keeps track of all combinations of your image processing
parameters. You can choose which parameters are used during processing in a later step.

Let's view the attributes and insert data into `imaging.ProcessingParamSet`.

In [ ]:
# insert clustering task manually
params_ks = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0,
}
ephys.ClusteringParamSet.insert_new_params(
    clustering_method="kilosort2",
    paramset_idx=0,
    params=params_ks,
    paramset_desc="Spike sorting using Kilosort2",
)
ephys.ClusteringParamSet()

Now that we've inserted suite2p parameters into the `ProcessingParamSet` table,
we're almost ready to run image processing. DataJoint uses a `ProcessingTask` table to
manage which `Scan` and `ProcessingParamSet` should be used during processing. 

This table is important for defining several important aspects of
downstream processing. Let's view the attributes to get a better understanding. 

In [ ]:
ephys.ClusteringTask.describe()

In [ ]:
ephys.ClusteringTask.heading

In [ ]:
ephys.ClusteringTask.insert1(
    dict(
        session_key,
        insertion_number=0,
        paramset_idx=0,
        clustering_output_dir="subject6/session1/towersTask_g0_imec0",
    ),
    skip_duplicates=True,
)

In [ ]:
ephys.Clustering.populate(display_progress=True)
ephys.CuratedClustering.populate(session_key, display_progress=True)
ephys.LFP.populate(session_key, display_progress=True)
ephys.WaveformSet.populate(session_key, display_progress=True)